In [1]:
!pip install simpletransformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import re

Fine Tuning of Simple Transformer

In [3]:
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs

C:\Users\Ayush Sinha\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_config=LanguageModelingArgs()

In [5]:
model_config.reprocess_input_data=True
model_config.overwrite_output_dir=True
model_config.num_train_epochs=3
model_config.best_model_dir="output/best_model"
model_config.save_best_model=True
model_config.train_batch_size=1
model_config.dataset_type="simple"
model_config.mlm=False
model_config.vocab_size=50257

Data Collection and Preparation

In [6]:
with open('aitext.txt','r') as fp:
    text = fp.read().split('\n')

In [7]:
text

['Artificial intelligence (AI) is the intelligence of machines or software, as opposed to the intelligence of human beings or animals.',
 'AI applications include advanced web search engines  recommendation systems, understanding human speech, self-driving cars, generative or creative tools (ChatGPT and AI art), and competing at the highest level in strategic games.',
 'Artificial intelligence was founded as an academic discipline in 1956. The field went through multiple cycles of optimism followed by disappointment and loss of funding, but after 2012, when deep learning surpassed all previous AI techniques,there was a vast increase in funding and interest.',
 "The various sub-fields of AI research are centered around particular goals and the use of particular tools. The traditional goals of AI research include reasoning, knowledge representation, planning, learning, natural language processing, perception, and support for robotics.General intelligence (the ability to solve an arbitrar

In [8]:
text[3].split('.')

['The various sub-fields of AI research are centered around particular goals and the use of particular tools',
 ' The traditional goals of AI research include reasoning, knowledge representation, planning, learning, natural language processing, perception, and support for robotics',
 "General intelligence (the ability to solve an arbitrary problem) is among the field's long-term goals",
 'To solve these problems, AI researchers have adapted and integrated a wide range of problem-solving techniques, including search and mathematical optimization, formal logic, artificial neural networks, and methods based on statistics, probability, and economics',
 'AI also draws upon psychology, linguistics, philosophy, neuroscience and many other fields',
 '']

In [9]:
bag = [txt for sentence in text for txt in sentence.split('.') if txt!='']
bag_size=len(bag)

In [10]:
bag_size

18

In [11]:
import random

In [12]:
train_sentence=[]
test_sentence=[]
label=[]
for p in text:
    sentences=[
        sentence for sentence in p.split('.') if sentence!=''
    ]
    num_sentence = len(sentences)
    if num_sentence>1:
        start = random.randint(0,num_sentence-2)
        if random.random()>=0.5:
            train_sentence.append(sentences[start])
            test_sentence.append(sentences[start+1])
            label.append(0)
        else:
            index=random.randint(0,bag_size-1)
            train_sentence.append(sentences[start])
            test_sentence.append(bag[index])
            label.append(1)

In [13]:
for i in range(4):
    print(label[i])
    print(train_sentence[i] + '\n---')
    print(test_sentence[i] + '\n---')

1
Artificial intelligence was founded as an academic discipline in 1956
---
Many of these algorithms are insufficient for solving large reasoning problems because they experience a "combinatorial explosion": they became exponentially slower as the problems grew larger
---
1
General intelligence (the ability to solve an arbitrary problem) is among the field's long-term goals
---
AI also draws upon psychology, linguistics, philosophy, neuroscience and many other fields
---
1
 These consist of particular traits or capabilities that researchers expect an intelligent system to display
---
 They solve most of their problems using fast, intuitive judgments
---
1
Early researchers developed algorithms that imitated step-by-step reasoning that humans use when they solve puzzles or make logical deductions
---
Many of these algorithms are insufficient for solving large reasoning problems because they experience a "combinatorial explosion": they became exponentially slower as the problems grew lar

In [14]:
len(train_sentence)

5

In [15]:
len(test_sentence)

5

Data Preprocessing

In [16]:
def textCleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

Train and Test

In [17]:
train_data = open('Train_data.txt','w')
text_data=textCleaning(train_sentence)
train_data.write(text_data)
train_data.close()

In [18]:
test_data = open('Test_data.txt','w')
text_data=textCleaning(test_sentence)
test_data.write(text_data)
test_data.close()

In [19]:
train_file='Train_data.txt'
test_file='Test_data.txt'

Choosing a Pretrained Transformer Model

In [20]:
model=LanguageModelingModel(
    "gpt2","gpt2",args=model_config, train_files=train_file,use_cuda=False
)

In [24]:
model.train_model(train_file,eval_first=train_file)

Epoch 3 of 3: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:32<00:00, 10.80s/it]


(3, 4.292177041371663)

In [25]:
eval_score=model.eval_model(train_file)

Running Evaluation: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.86it/s]


Loss and Perplexity

In [26]:
eval_score

{'eval_loss': 3.415811061859131, 'perplexity': tensor(30.4416)}

Model Training

In [ ]:
from simpletransformers.language_generation import LanguageGenerationModel, LanguageGenerationArgs

In [ ]:
lang_gen_args = LanguageGenerationArgs()
lang_gen_args.max_length=1

In [ ]:
# from simpletransformers.language_generation.language_generation_model import MAX_LENGTH
model = LanguageGenerationModel("gpt2","/content/outputs/checkpoint-1-epoch-1",use_cuda=False)
output = model.generate("Have a nice ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [ ]:
model_name="gpt2-medium"

In [ ]:
model=GPT2LMHeadModel.from_pretrained(model_name)
tokenizer=GPT2Tokenizer.from_pretrained(model_name,pad_token='<PAD>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.add_special_tokens({'pad_token':'[PAD]'})

1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
tokenized_train_data = [tokenizer.encode(text, return_tensors="pt").to(device) for text in train_file]

In [ ]:
text='Hows your day '
text_ids=tokenizer.encode(text, return_tensors="pt").to(device)

In [ ]:
outputs = model.generate(text_ids, max_length=10, num_return_sequences=1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated_text=tokenizer.decode(outputs[0],skip_special_tokens=True)

In [ ]:
generated_text.replace('\xa0',"")
generated_text.replace('\n',"")

'Train_data.txtThe file is'

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

Fine Tuning and Hyperparameter Tuning

In [ ]:
def load_model(model_name):
  model=GPT2LMHeadModel.from_pretrained(model_name)
  return model

In [ ]:
def load_tokenizer(tokenizer_path):
  tokenizer=GPT2Tokenizer.from_pretrained(tokenizer_path)
  return tokenizer

In [ ]:
def generate_text(sequence, max_length):
  model_name="gpt2-medium"
  model=load_model(model_name)
  tokenizer=load_tokenizer(model_name)
  ids=tokenizer.encode(f"{sequence}", return_tensors='pt')
  final_outputs=model.generate(
      ids,
      do_sample=True,
      max_length=max_length,
      pad_token_id=model.config.eos_token_id,
      top_k=50,
      top_p=0.95)
  print(tokenizer.decode(final_outputs[0],skip_special_tokens=True))

Inference

In [ ]:
sequence=input()
max_len=int(input())
generate_text(sequence,max_len)

what are you
8
what are you thinking you're doing?"


In [ ]:
validation_texts=['Artificial intelligence (AI) is the intelligence of machines or software, as opposed to the intelligence of human beings or animals.','Early researchers developed algorithms that imitated step-by-step reasoning that humans use when they solve puzzles or make logical deductions. By the late 1980s and 1990s, methods were developed for dealing with uncertain or incomplete information, employing concepts from probability and economics.']

Success Metrics

In [ ]:
perplexities=[]
for text in validation_texts:
  input_ids=tokenizer.encode(text,return_tensors='pt')
  with torch.no_grad():
    loss=model(input_ids,labels=input_ids).loss
    perplexities.append(torch.exp(loss))

average_perplexity = sum(perplexities)/len(perplexities)
print("Average Perplexity: ", average_perplexity.item())

Average Perplexity:  20.070371627807617
